## Intro To Biocomputing Final -- Winter 2017
Structure is a Bayesian clustering algorithm commonly used in population genetic studies (Pritchard et al., 2000). In fact, this program has been cited more than 10k times in the scientific literature (Puechmaille, 2016). Inferring the genetic structure of subpopulations is an important facet of many laboratories spanning multiple fields of evolutionary and ecological studies. The interest of subpopulations, or K-clusters, in genetic data has given rise to multiple methods of K-estimation(Puechmaille, 2016). During my Master's studies, I used microsatellite markers to capture genetic diversity in populations of a wild mountain flower of the North American west known as Gunnison's Sego Lily (Calochortus gunnisonii). Using the population clustering algorithm of the Structure program (Pritchard et al., 2000), combined with that of the highly popular Evanno Method (4,000+ citations; Puechmaille, 2016) as a K estimator, my data suggests a value of K=2 and further evidence of K=5 (Fuller and McGlaughlin, unpublished). However, some disparate sampling exists between populations, and Puechmaille, 2016 presents and tests four new methods for K-estimation in simulated and empirical datasets. His results suggest that these new methods may perform more accurately as estimators of K-clusters than the Evanno method when sampling is uneven. Using the supplementary materials of Puechmaille as a guide, I have embarked upon my first coding project to automate this K-estimation using Python. An R-code does exist and is written by Sebastian Puechmaille (find it here: http://batlab.ucd.ie/~spuechmaille/).

The following is code written to import Structure Results text files, cut out the block of data desired, and iterate/analyze all files in a user's 'Results' directory. This code is able to detect the number user-defined populations, K-clusters, and iterations -- providing an output for the MaxMean method created by Sebastian Puechmaille, 2016. It is suggested that researchers exhaustively examine their genetic data and clustering landscape by utilizing multiple K-means methods and ensure their conclusions are biologically realistic given their organism's natural history (Puechmaille, 2016).

Please find four compilations of code in this GitHub directory that estimate K. Useful guidance comments have been included in the coding block to assist the user. For my data involving C. gunnisonii, the 'MaxMean' method produced a best value of K=8. This will require further testing and vetting against other K-cluster algorithms.

*Note that in the concatenation of the Dataframes from Pandas, there is an autosort function with concat that sorts lexicographically if the users table columns don't match, meaning the indices (table column headers) are not in numerical order. This problem has been identified by the GitHub/Overstack communities but remains an open problem. See issue here on GitHub (https://github.com/pandas-dev/pandas/issues/4588). One can explore this in my code by running the output of the 'ar_concat' object below.  

In [ ]:
#Import the necessary packages to the jupyter platform
import csv
import re
import pandas as pd
import numpy as np
import glob

In [ ]:
def import_df(filename):
    rows = []
    with open(filename) as data:
        for line in data:
            if line.startswith("Inferred ancestry of individuals"):
                # skip header of table
                next(data)
                break
        #Reads text until the end of the block:
        k = -1 #set the value  
        for line in data:
            if not line.strip():#If Read empty line, break at end of table
                break
            words = line.split()
            if k < 0 :
                k = len(words)- words.index(':') - 1

            d = dict(ind=words[1], pop=int(words[3]))
            for i in range(k):
                key = 'pop' + str(i)
                value = float(words[5+i])
                d[key] = value
            rows.append(d)
    
    #Create a dataframe from the 'rows' list of dictionaries
    #Will have format of: User Pop/Ind, PopId, and 
    #Population Proportion for each proposed K-value
    df = pd.DataFrame(rows)

    #Group the new Dataframe by PopId and take means for each population
    df1 = df.groupby('pop', as_index = True).mean()

    #Create an empty dataframe 'df2'
    df2 = pd.DataFrame(dtype = int)

    #Find the Maxiumum Mean for each K-value over all Populations
    Kmeanmax = df1.max(axis = 0)

    #Append the empty dataframe with the Max Means of each K-value
    df2 = df2.append(Kmeanmax, ignore_index = True)

    #Assign the column names (i.e., pop0, pop1,..popN) to object 'cols'
    cols = df2.columns

    #Add a column to show original # of K-values tested for a certain file 
    df2['K_tested'] = df2.count(axis = 1)

    #Test each Max Mean of each K-value against a threshold value (user-defined) 
    #And sum the K-values that meet the threshhold, disgarding the ones that don't
    #Make the ouput a new column at the end of the dataframe
    df2['K_Actual'] = (df2[cols].values > 0.5).sum(axis = 1)

    df2['Filename'] = filename
    return df2

In [ ]:
allres = []
for ff in glob.glob("../data/Results.copy.NoK1/*_f"):
    allres.append(import_df(ff))

In [ ]:
#Create a list of dataframes
#allres = []
#for ff in glob.glob("../data/Results.copy.NoK1/*_f"):
    #allres.append(import_df(ff))

#Since all of the files in your Results Folder are now in a list with defined threshholds, and testing
#against those thresholds, we can concatenate all of the dataframes together and begin to parse the data
ar_concat = pd.concat(allres)

#Now we will sort values in the table on K-tested column, which is what we want
##Note, the proportion of individual membership columns get automatically sorted with the
#function 'concatenate' -- this sorts lexicographically (alphabetically, instead of numerically) and raises 
#issues while parsing the data -- note that if user wants to see the proportions for each K-value over each grouping
#of K, it is not trivial at this time

#this probelem has been noticed on github, yet there is no solution at the current time -- problems likely
#lie in the way I have input data that create the indices for the DataFrames -- online tutorials say I need 
#to convert my indices into objects to be sorted numerically
ar_concat = ar_concat.sort_values('K_tested')

#We can now 'groupby' the K-tested column (splits into groups of user-defined K-values) and take
#max for 'K-Actual' column -- providing us our K-estimate of this Results folder! 
groupby_K_tested = ar_concat['K_Actual'].groupby(ar_concat['K_tested']).max()

#Define new object for the K-value estimate output and write to a file
MaxMeanK = groupby_K_tested.max()

#MaxMeanKdf = pd.DataFrame(dtype = int)

#MaxMeanKdf = MaxMeanKdf.append(MaxMeanK, ignore_index = True)

MaxMeanK



